# Work 7: Pegasus-large fine-tuned on Extracted Summaries

Here we fine tune on summaries extracted from the 'claims' and 'description' sections.

.... this little notebook never got her day in the sun because notebook 4 never came through with the data.. oh well, another day.

Fine tuning code reference: HuggingFace [Transformers Notebooks](https://github.com/huggingface/transformers/tree/main/notebooks)

## Setup

In [ ]:
#install libraries
!pip install -q datasets
!pip install -q transformers
!pip install --quiet --upgrade accelerate
!pip install -q sentencepiece
!pip install -q evaluate
!pip install -q rouge_score
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Install Packages

#standard data science libraries
import pandas as pd
import numpy as np
import random

#visualization
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import display, HTML

#datasets
import datasets
from datasets import load_from_disk
from datasets import load_dataset, load_metric

#transformers
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TrainingArguments
from transformers import AutoTokenizer

# PyTorch
import torch

#rouge
import evaluate
from evaluate import load

#nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
##### ensure you mount to the folder that you want.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_save_dir="/content/drive/MyDrive/W266/HUPD/"
model_save_dir="/content/drive/MyDrive/W266/ProjectModels/Pegasus_large_ft_extractive/"

Mounted at /content/drive


##Load Data

In [ ]:
##########################for the test on small data
dataset = load_from_disk(data_save_dir + 'extracted_train_tokenized')
val_presplit = load_from_disk(data_save_dir + 'extracted_val_tokenized')

In [ ]:
dataset

Dataset({
    features: ['patent_number', 'decision', 'title', 'abstract', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'claims_desc', 'extractive_summaries', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 576
})

In [ ]:
# val_test_split = dataset['validation'].train_test_split(test_size=0.5)

In [ ]:
# dataset['train'] = dataset['train']
# dataset['validation'] = val_test_split['train']
# dataset['test'] = val_test_split['test']

In [ ]:
# dataset

In [ ]:
#only run if you want to shrink the amount of data you are working with.
# train_set = train_set.shuffle(seed=42).select(range(1000))
# val_set = val_set.shuffle(seed=42).select(range(100))
# test_set = test_set.shuffle(seed=42).select(range(100))

# Modeling

In [ ]:
model_checkpoint = "google/pegasus-large"

In [ ]:
metric = load('rouge')

### Tokenize Data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
max_input_length = 500 #any longer and we run out of memory
max_summary_length = 100

In [ ]:
def preprocess_text(dataset, summarize_section = 'description'):
  inputs = dataset[summarize_section]
  model_inputs  = tokenizer(inputs, truncation = True, max_length = max_input_length) #, return_tensors = 'pt', , padding = 'max_length'
  labels = dataset['abstract']
  model_labels = tokenizer(text_target = labels, truncation = True, max_length = max_input_length) #, return_tensors = 'pt' , padding = 'max_length'
  model_inputs['labels'] = model_labels['input_ids']
  return model_inputs

In [ ]:
# tokenized_dataset = dataset.map(preprocess_text, batched=True)

Map:   0%|          | 0/30915 [00:00<?, ? examples/s]

Map:   0%|          | 0/3655 [00:00<?, ? examples/s]

Map:   0%|          | 0/3656 [00:00<?, ? examples/s]

In [ ]:
# tokenized_dataset.save_to_disk(data_save_dir + 'HUPD_C07_tokenized_descriptions')

Saving the dataset (0/12 shards):   0%|          | 0/30915 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/3655 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/3656 [00:00<?, ? examples/s]

### Set up Model and Fine Tune

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-descriptions",
    evaluation_strategy = "epoch",
    learning_rate=10e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=6,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value for key, value in result.items()} #(took out *100)

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
#plot two dictionarries as two lines, one with structure {epoch: eval_loss} the other with structure {epoch:loss}. Plot both lines on the same chart.

# val_loss = {trainer.state.log_history[i]['epoch']:trainer.state.log_history[i]['eval_loss'] for i in [  ]}
# train_loss = {trainer.state.log_history[i]['epoch']:trainer.state.log_history[i]['loss'] for i in [  ]}
# plt.plot(train_loss.keys(), train_loss.values(), label = train_loss)
# plt.plot(val_loss.keys(), val_loss.values(), label = val_loss)
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Pegasus-large fine-tuned on Extracted HUPD Sample')
# plt.legend(['Train Loss', 'Eval Loss'])
# plt.show()


In [ ]:
trainer.evaluate()

In [ ]:
test_predictions = trainer.predict(
                test_dataset = tokenized_dataset['test'],
                num_beams = 12,
                max_length = max_summary_length
                )

In [ ]:
test_predictions.metrics

In [ ]:
summarized_texts = []
for summary in test_predictions.predictions:
    summarized_text = tokenizer.batch_decode(summary, skip_special_tokens=False)
    summarized_texts.append(summarized_text)

In [ ]:
num_examples = 10
random_indices = np.random.choice(len(tokenized_dataset['test']), num_examples, replace=False)
print(random_indices)

In [ ]:
# Randomly sample example paies
sampled_summarized_texts = [summarized_texts[i] for i in random_indices]
sampled_abstracts = [tokenized_dataset['test']['abstract'][i] for i in random_indices]

# Zip the lists together
comparisons = list(zip(sampled_summarized_texts, sampled_abstracts))

# Create a pandas DataFrame from the zipped lists
comparison_df = pd.DataFrame(comparisons, columns=['predicted_summary', 'abstract'])

# Display df
display(HTML(comparison_df.to_html()))

In [ ]:
# del model
# del trainer
# torch.cuda.empty_cache()

## Check How It Worked on Test Data

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
num_examples = 10
random_indices = np.random.choice(len(tokenized_dataset['test']), num_examples, replace=False)
print(random_indices)

In [ ]:
#specify compute device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#set format to Torch
tokenized_dataset['test'].set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
batch_size = 10 #note that the standard batch size for the DataLoader is 200
test_dataloader = DataLoader(tokenized_dataset['test'], batch_size = batch_size)

In [ ]:
summarized_texts = []
for batch in iter(test_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    summary = model.generate(input_ids = input_ids,
                             attention_mask = attention_mask,
                             max_length = 100,
                            #  num_beams = 10
                             )
    summarized_text = tokenizer.batch_decode(summary, skip_special_tokens=False)
    summarized_texts.extend(summarized_text)

In [ ]:
#Generate Rouge Scores
rouge = evaluate.load('rouge')
predictions = summarized_texts
references = tokenized_dataset['test']['abstract']
claims_results = rouge.compute(predictions=predictions,
                        references=references)
print(claims_results)

In [ ]:
# Randomly sample example paies
sampled_summarized_texts = [summarized_texts[i] for i in random_indices]
sampled_abstracts = [tokenized_dataset['test']['abstract'][i] for i in random_indices]

# Zip the lists together
comparisons = list(zip(sampled_summarized_texts, sampled_abstracts))

# Create a pandas DataFrame from the zipped lists
comparison_df = pd.DataFrame(comparisons, columns=['predicted_summary', 'abstract'])

# Display df
display(HTML(comparison_df.to_html()))